<a href="https://colab.research.google.com/github/ankan-git-coder/Deep-learning-COURSE/blob/main/cnn_mri_test_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(90),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [25]:
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [26]:
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/alzheimer dataset/train', transform=train_transform)
test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/alzheimer dataset/test', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [27]:
class AlzheimarCNN(nn.Module):
  def __init__(self):
      super(AlzheimarCNN, self).__init__()
      self.layer1 = nn.Sequential(
          nn.Conv2d(1,256,kernel_size=3,padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2)
      )
      self.layer2= nn.Sequential(
          nn.Conv2d(256,64,kernel_size=3,padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2)
      )
      self.layer3 = nn.Sequential(nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2))
      self.layer4 = nn.Sequential(nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2))
      self.layer5 = nn.Sequential(nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2))
      self.flatten =nn.Flatten()
      self.fc1=nn.Linear(64*4*4,128)
      self.dropout=nn.Dropout(0.2)
      self.fc2=nn.Linear(128, 64)
      self.output = nn.Linear(64, 4)

  def forward(self, x):
      x = self.layer1(x)
      x = self.layer2(x)
      x = self.layer3(x)
      x = self.layer4(x)
      x = self.layer5(x)
      x = self.flatten(x)
      x = torch.relu(self.fc1(x))
      x = self.dropout(x)
      x = torch.relu(self.fc2(x))
      x = self.output(x)
      return x

model = AlzheimarCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/50, Loss: 1.0183
Epoch 2/50, Loss: 0.9334
Epoch 3/50, Loss: 0.9189
Epoch 4/50, Loss: 0.8863
Epoch 5/50, Loss: 0.8783
Epoch 6/50, Loss: 0.8507
Epoch 7/50, Loss: 0.8198
Epoch 8/50, Loss: 0.8076
Epoch 9/50, Loss: 0.7821
Epoch 10/50, Loss: 0.7752
Epoch 11/50, Loss: 0.7726
Epoch 12/50, Loss: 0.7617
Epoch 13/50, Loss: 0.7570
Epoch 14/50, Loss: 0.7470
Epoch 15/50, Loss: 0.7434
Epoch 16/50, Loss: 0.7423
Epoch 17/50, Loss: 0.7328
Epoch 18/50, Loss: 0.7235
Epoch 19/50, Loss: 0.7228
Epoch 20/50, Loss: 0.7076
Epoch 21/50, Loss: 0.7047
Epoch 22/50, Loss: 0.7156
Epoch 23/50, Loss: 0.6975
Epoch 24/50, Loss: 0.6910
Epoch 25/50, Loss: 0.6816
Epoch 26/50, Loss: 0.6832
Epoch 27/50, Loss: 0.6792
Epoch 28/50, Loss: 0.6648
Epoch 29/50, Loss: 0.6810
Epoch 30/50, Loss: 0.6704
Epoch 31/50, Loss: 0.6645
Epoch 32/50, Loss: 0.6656
Epoch 33/50, Loss: 0.6574
Epoch 34/50, Loss: 0.6421
Epoch 35/50, Loss: 0.6537
Epoch 36/50, Loss: 0.6362
Epoch 37/50, Loss: 0.6377
Epoch 38/50, Loss: 0.6322
Epoch 39/50, Loss: 0.

In [28]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total}%')

Test Accuracy: 22.61445118587976%
